In [1]:
import pandas as pd
base = pd.read_csv('house_prices.csv')
base.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [2]:
base.shape

(21613, 21)

In [3]:
X = base.iloc[:, 5:6]
y = base.iloc[:, 2:3]

In [4]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
X = scaler_x.fit_transform(X)
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y)

In [5]:
import tensorflow

In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
columns = [tf.feature_column.numeric_column('x', shape=[1])]
columns

[NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [8]:
regressor = tf.estimator.LinearRegressor(feature_columns=columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp18izfsy9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [10]:
X_train.shape

(15129, 1)

In [11]:
X_test.shape

(6484, 1)

In [12]:
func_training = tf.estimator.inputs.numpy_input_fn({'x': X_train}, y_train, batch_size=32, num_epochs=None, shuffle=True)

In [13]:
func_test = tf.estimator.inputs.numpy_input_fn({'x': X_test}, y_test, batch_size=32, num_epochs=1000, shuffle=False)

In [16]:
regressor.train(input_fn=func_training, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp18izfsy9/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 20000...
INFO:tensorflow:Saving checkpoints for 20000 into /tmp/tmp18izfsy9/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 20000...
INFO:tensorflow:loss = 22.740007, step = 20001
INFO:tensorflow:global_step/sec: 706.859
INFO:tensorflow:loss = 6.7211494, step = 20101 (0.143 sec)
INFO:tensorflow:global_step/sec: 806.414
INFO:tensorflow:loss = 10.475578, step = 20201 (0.123 sec)
INFO:tensorflow:global_step/sec: 709.125
INFO:tensorflow:loss = 7.8237753, step = 20301 (0.146 sec)
INFO:tensorflow:global_step/sec: 763.303
INFO:tensorflow:loss = 8.205025, step = 20401 (0.126 sec)
INFO:

In [17]:
# Para visualizar métricas
metrics_training = regressor.evaluate(input_fn=func_training, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-10-14T20:17:41Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp18izfsy9/model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Inference Time : 17.71253s
INFO:tensorflow:Finished evaluation at 2020-10-14-20:17:59
INFO:tensorflow:Saving dict for global step 30000: average_loss = 0.5087494, global_step = 30000, label/mean = -0.00090695947, loss = 16.279982, prediction/mean = -0.00

In [18]:
metrics_test = regressor.evaluate(input_fn=func_test, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-10-14T20:18:34Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp18izfsy9/model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Inference Time : 17.55002s
INFO:tensorflow:Finished evaluation at 2020-10-14-20:18:52
INFO:tensorflow:Saving dict for global step 30000: average_loss = 0.50347424, global_step = 30000, label/mean = 0.0026366194, loss = 16.111176, prediction/mean = 0.0021

In [19]:
metrics_training

{'average_loss': 0.5087494,
 'label/mean': -0.00090695947,
 'loss': 16.279982,
 'prediction/mean': -0.007547772,
 'global_step': 30000}

In [20]:
metrics_test

{'average_loss': 0.50347424,
 'label/mean': 0.0026366194,
 'loss': 16.111176,
 'prediction/mean': 0.0021081246,
 'global_step': 30000}

In [23]:
import numpy as np
new_homes = np.array([[800], [900], [1000]])

In [24]:
new_homes = scaler_x.transform(new_homes)
new_homes

array([[-1.39358923],
       [-1.28470655],
       [-1.17582386]])

In [25]:
func_prediction = tf.estimator.inputs.numpy_input_fn({'x': new_homes}, shuffle=False)
predict = regressor.predict(input_fn=func_prediction)

In [26]:
list(predict)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp18izfsy9/model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([-0.9909516], dtype=float32)},
 {'predictions': array([-0.9139028], dtype=float32)},
 {'predictions': array([-0.83685386], dtype=float32)}]

In [27]:
for p in regressor.predict(input_fn=func_prediction):
    print(scaler_y.inverse_transform(p['predictions']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp18izfsy9/model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[176291.27]
[204577.33]
[232863.42]
